In [1]:
import polars as pl
import pandas as pd
import numpy as np
import pyarrow as pa
import plotly.express as px
import string
import random
from datetime import datetime

# Motivation

1. Small memory footpring
  - Native dtypes: missing, strings.
  - Arrow format.
1. Query Planning
1. Parallelism:
    - Speed
    - Debugging




## Memory Footprint


### Memory Footprint of Storage

Polars vs. Pandas:

In [2]:
letters = pl.Series(list(string.ascii_letters))

n = int(10e6)
letter1 = letters.sample(n,with_replacement=True)
letter1.estimated_size(unit='gb')

0.08381903916597366

In [3]:
letter1_pandas = letter1.to_pandas() 
letter1_pandas.memory_usage(deep=True, index=False) / 1e9

0.58

The memory footprint of the polars Series is 1/7 of the pandas Series(!).
But I did cheat- I used string type data to emphasize the difference. The difference would have been smaller if I had used integers or floats. 




### Memory Footprint of Compute

You are probably storing your data to compute with it.
Let's compare the memory footprint of computations. 


In [4]:
%load_ext memory_profiler

In [5]:
%memit letter1.sort()

peak memory: 551.62 MiB, increment: 222.48 MiB


In [6]:
%memit letter1_pandas.sort_values()

peak memory: 689.50 MiB, increment: 359.88 MiB


In [7]:
%memit letter1[10]='a'

peak memory: 497.78 MiB, increment: 81.95 MiB


In [8]:
%memit letter1_pandas[10]='a'

peak memory: 416.96 MiB, increment: 0.00 MiB


Things to notice:

- Operating on existing data consumes less memory in polars than in pandas.
- Changing the data consumes more memory in polars than in pandas. Why is that?


### Operating From Disk to Disk

What if my data does not fit into RAM?
Turns out you can read from disk, process in RAM, and write to disk. This allows you to process data larger than your memory. 

TODO: demonstrate sink_parquet from [here](https://www.rhosignal.com/posts/sink-parquet-files/).





## Query Planning

Consider a sort opperation that follows a filter operation. 
Ideally, filter precededs the sort, but we did not ensure this...
We now demonstarte that polars' query planner will do it for you. 
En passant, we see polars is more efficient also without the query planner. 


Polars' Eager evaluation, without query planning. 
Sort then filter. 

In [9]:
%timeit -n 2 -r 2 letter1.sort().filter(letter1.is_in(['a','b','c']))

676 ms ± 31.4 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Polars' Eager evaluation, without query planning. 
Filter then sort. 

In [10]:
%timeit -n 2 -r 2 letter1.filter(letter1.is_in(['a','b','c'])).sort()

207 ms ± 3.8 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Polars' Lazy evaluation with query planning. 
Recieves sort then filter; executes filter then sort. 

In [11]:
%timeit -n 2 -r 2 letter1.alias('letters').to_frame().lazy().sort(by='letters').filter(pl.col('letters').is_in(['a','b','c'])).collect()

179 ms ± 1.71 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Pandas' eager evaluation in the wrong order: Sort then filter. 

%timeit -n 2 -r 2 letter1_pandas.sort_values().loc[lambda x: x.isin(['a','b','c'])]
```


Pandas eager evaluation in the right order: Filter then sort. 

In [12]:
%timeit -n 2 letter1_pandas.loc[lambda x: x.isin(['a','b','c'])].sort_values()

683 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 2 loops each)


Pandas alternative syntax, just as slow. 

In [13]:
%timeit -n 2 -r 2 letter1_pandas.loc[letter1_pandas.isin(['a','b','c'])].sort_values()

719 ms ± 14.2 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Things to note:

1. Query planning works!
1. Polars faster than Pandas even in eager evaluation (without query planning).



## Parallelism

Polars seamlessly parallelizes over columns (also within, when possible).
As the number of columns in the data grows, we would expect fixed runtime until all cores are used, and then linear scaling.
The following code demonstrates this idea, using a simple sum-within-column.


In [14]:
import time

def scaling_of_sums(n_rows, n_cols):
  # n_cols = 2
  # n_rows = int(1e6)
  A = {}
  A_numpy = np.random.randn(n_rows,n_cols)
  A['numpy'] = A_numpy.copy()
  A['polars'] = pl.DataFrame(A_numpy)
  A['pandas'] = pd.DataFrame(A_numpy)

  times = {}
  for key,value in A.items():
    start = time.time()
    value.sum()
    end = time.time()
    times[key] = end-start

  return(times)

In [15]:
scaling_of_time = {
  p:scaling_of_sums(n_rows= int(1e6),n_cols = p) for p in np.arange(1,16)}

In [16]:
data = pd.DataFrame(scaling_of_time).T
px.line(
  data, 
  labels=dict(
    index="Number of Columns", 
    value="Runtime")
)

Things to note:

- Pandas is slow. 
- Numpy is quite efficient.
- My machine has 8 cores. I would thus expect a fixed timing until 8 columns, and then linear scaling. This is not the case. I wonder why?


## Speed Of Import

Polar's `read_x` functions are quite faster than Pandas. 
This is due to better type "guessing" heuristics, and to native support of the parquet file format. 

We now make synthetic data, save it as csv or parquet, and reimport it with polars and pandas.

Starting with CSV:

In [17]:
n_rows = int(1e5)
n_cols = 10
data = np.random.randn(n_rows,n_cols)
data.tofile('data/data.csv', sep = ',')

Import with pandas. 

In [18]:
%timeit -n2 -r2 data_pandas = pd.read_csv('data/data.csv', header = None)

19.2 s ± 421 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Import with polars. 

In [19]:
%timeit -n2 -r2 data_polars = pl.read_csv('data/data.csv', has_header = False)

2.86 s ± 99.2 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Moving to parquet:


In [20]:
data_pandas = pd.DataFrame(data)
data_pandas.columns = data_pandas.columns.astype(str)
data_pandas.to_parquet('data/data.parquet', index = False)

In [21]:
%timeit -n2 -r2 data_pandas = pd.read_parquet('data/data.parquet')

15.9 ms ± 4.21 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


In [22]:
%timeit -n2 -r2 data_polars = pl.read_parquet('data/data.parquet')

9 ms ± 1.79 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Things to note:

- The difference in speed is quite large.
- I dare argue that polars' type guessing is better, but I am not demonstrating it here. 
- Bonus fact: parquet is much faster than csv, and also saves the frame's schema.



## Speed Of Join

Because pandas is built on numpy, people see it as both an in-memory database, and a matrix/array library.
With polars, it is quite clear it is an in-memory database, and not an array processing library.
As such, you cannot multiply two polars dataframes, but you can certainly join then efficiently.

Make some data:

In [23]:
def make_data(n_rows, n_cols):
  data = np.concatenate(
  (
    np.arange(n_rows)[:,np.newaxis], # index
    np.random.randn(n_rows,n_cols), # values
    ),
    axis=1)
    
  return data


n_rows = int(1e6)
n_cols = 10
data_left = make_data(n_rows, n_cols)
data_right = make_data(n_rows, n_cols)

Polars join:

In [24]:
data_left_polars = pl.DataFrame(data_left)
data_right_polars = pl.DataFrame(data_right)

%timeit -n2 -r2 polars_joined = data_left_polars.join(data_right_polars, on = 'column_0', how = 'inner')

228 ms ± 37.1 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


Pandas join:

In [25]:
data_left_pandas = pd.DataFrame(data_left)
data_right_pandas = pd.DataFrame(data_right)

%timeit -n2 -r2 pandas_joined = data_left_pandas.merge(data_right_pandas, on = 0, how = 'inner')

653 ms ± 105 ms per loop (mean ± std. dev. of 2 runs, 2 loops each)


## Moving Forward...

If this motivational seection has convinced you to try polars instead of pandas, here is a  more structured intro. 






# Polars Series

Much like pandas, polars' fundamental building block is the series. 
A series is a column of data, with a name, and a dtype.
In the following we:

1. Create a series and demonstrate basic operations on it.
1. Demonstrate the various dtypes. 
1. Discuss missing values.
1. Filter a series.

## Object Housekeeping
Construct a series

In [26]:
s = pl.Series("a", [1, 2, 3])
s

a
i64
1
2
3


Make pandas series for comparison:

In [27]:
s_pandas = pd.Series([1, 2, 3], name = "a")

In [28]:
type(s)

polars.internals.series.series.Series

In [29]:
type(s_pandas)

pandas.core.series.Series

In [30]:
s.dtype

Int64

In [31]:
s_pandas.dtype

dtype('int64')

Renaming a series; will be very useful when operating on dataframe columns.

In [32]:
s.alias("b")

b
i64
1
2
3


In [33]:
s.clone()

a
i64
1
2
3


In [34]:
s.clone().append(pl.Series("a", [4, 5, 6]))

a
i64
1
2
3
4
5
6


Note: `series.append` operates in-place. That is why we cloned the series first.

Flatten a list of lists using `explode()`.

In [35]:
pl.Series("a", [[1, 2], [3, 4], [9, 10]]).explode()

a
i64
1
2
3
4
9
10


In [36]:
s.extend_constant(666, n=2)

a
i64
1
2
3
666
666


In [37]:
#| eval: false
s.new_from_index()

In [38]:
s.rechunk()

a
i64
1
2
3


In [39]:
s.rename("b", in_place=False) # has an in_place option. Unlike .alias()

b
i64
1
2
3


In [40]:
s.to_dummies()

a_1,a_2,a_3
u8,u8,u8
1,0,0
0,1,0
0,0,1


In [41]:
s.cleared() # creates an empty series, with same dtype

a
i64


Consturcting a series of floats, for later use.

In [42]:
f = pl.Series("a", [1., 2., 3.])
f

a
f64
1.0
2.0
3.0


In [43]:
f.dtype

Float64

## Memory Representation of Series

Object size in memory. Super useful for profiling:

In [44]:
s.estimated_size(unit="gb")

2.2351741790771484e-08

In [45]:
s.chunk_lengths() # what is the length of each memory chunk?

[3]

## Filtering and Subsetting


In [46]:
s[0]

1

Filtering with boolneas requires a series of booleans, not a list:

In [47]:
s.filter(pl.Series("a", [True, False, True])) # works

a
i64
1
3


Will not work:

In [48]:
#| eval: false

s[[True, False, True]]

Don't be confused with pandas!

In [49]:
#| eval: false

s.loc[[True, False, True]] 

In [50]:
s.head(2)

a
i64
1
2


In [51]:
s.limit(2)

a
i64
1
2


Negative indexing is not supported:

In [52]:
#| eval: false

s.head(-1)
s.limit(-1)

In [53]:
s.tail(2)

a
i64
2
3


In [54]:
s.sample(2, with_replacement=False)

a
i64
1
3


In [55]:
s.take([0, 2]) # same as .iloc

a
i64
1
3


In [56]:
s.slice(1, 2) # same as pandas .iloc[1:2]

a
i64
2
3


In [57]:
s.take_every(2)

a
i64
1
3


## Aggregations

In [58]:
s.sum()

6

In [59]:
s.min()

1

In [60]:
s.arg_min()

0

In [61]:
s.mean()

2.0

In [62]:
s.median()

2.0

In [63]:
s.entropy()

-4.68213122712422

In [64]:
s.describe()

statistic,value
str,f64
"""min""",1.0
"""max""",3.0
"""null_count""",0.0
"""mean""",2.0
"""std""",1.0
"""count""",3.0


In [65]:
s.value_counts()

a,counts
i64,u32
2,1
1,1
3,1


## Object Transformations


In [66]:
pl.Series("a",[1,2,3,4]).reshape(dims = (2,2))

a
list[i64]
"[1, 2]"
"[3, 4]"


In [67]:
s.shift(1)

a
i64
null
1
2


In [68]:
s.shift(-1)

a
i64
2
3
null


In [69]:
s.shift_and_fill(1, 999)

a
i64
999
1
2


## Mathematical Transformations

In [70]:
s.abs()

a
i64
1
2
3


In [71]:
s.sin()

a
f64
0.841471
0.909297
0.14112


In [72]:
s.exp()

a
f64
2.718282
7.389056
20.085537


In [73]:
s.hash()

a
u64
13321499719149775801
8196255364589999986
3071011010030224171


In [74]:
s.log()

a
f64
0.0
0.693147
1.098612


In [75]:
s.peak_max()

""
bool
false
false
true


In [76]:
s.sqrt()

a
f64
1.0
1.414214
1.732051


In [77]:
s.clip_max(2)

a
i64
1
2
2


In [78]:
s.clip_min(1)

a
i64
1
2
3


You cannot round integers, but you can round floats.


In [79]:
f.round(2)

a
f64
1.0
2.0
3.0


In [80]:
f.ceil()

a
f64
1.0
2.0
3.0


In [81]:
f.floor()

a
f64
1.0
2.0
3.0


In [82]:
s.is_in(pl.Series([1, 10]))

a
bool
true
false
false


__Caution__: `is_in()` in polars has an underscore, unlike `isin()` in pandas.



## Cummulative Operations


In [83]:
s.cummax()

a
i64
1
2
3


In [84]:
s.cumsum()

a
i64
1
3
6


In [85]:
s.cumprod()

a
i64
1
2
6


In [86]:
s.ewm_mean(com=0.5)

a
f64
1.0
1.75
2.615385


## Sequential Operations


In [87]:
s.diff()

a
i64
null
1
1


In [88]:
s.pct_change()

a
f64
null
1.0
0.5


## Windowed Operations


In [89]:
s.rolling_apply(
  pl.sum, 
  window_size=2)

a
i64
null
3
5


Not all functions will work within a `rolling_apply`! Only polars' functions will.

In [90]:
#| eval: false

s.rolling_apply(np.sum, window_size=2) # will not work

In [91]:
s.rolling_max(window_size=2)

a
i64
null
2
3


In [92]:
s.clip(1, 2)

a
i64
1
2
2


In [93]:
s.clone()

a
i64
1
2
3


In [94]:
# check equality with clone
s == s.clone()

a
bool
true
true
true


## Binary Operations

Despite my introduction above, you still can think of polars series as 1D arrays...

In [95]:
s.dot(pl.Series("b", [1, 2, 3]))

14.0

## Uniques and Duplicates


In [96]:
s.is_duplicated()

a
bool
false
false
false


In [97]:
s.is_unique()

a
bool
true
true
true


In [98]:
s.n_unique()

3

In [99]:
pl.Series([1,2,3,4,1]).unique_counts()

""
u32
2
1
1
1


The first appearance of a value in a series:

In [100]:
pl.Series([1,2,3,4,1]).is_first()

""
bool
true
true
true
true
false


## dtypes

__Note__. Unlike pandas, polars' test functions have an underscore: `is_numeric()` instead of `isnumeric()`.


### Testing

In [101]:
s.is_numeric()

True

In [102]:
s.is_float()

False

In [103]:
s.is_utf8()

False

In [104]:
s.is_boolean()

False

In [105]:
s.is_datelike()

False

Compare with Pandas Type Checkers:

In [106]:
pd.api.types.is_string_dtype(s_pandas)

False

In [107]:
pd.api.types.is_string_dtype(s)

False

### Casting


In [108]:
s.cast(pl.Int32)

a
i32
1
2
3


Things to note: 

- `s.cast()` is an in place operation. If you want to keep the original series, you can use `s.cast(pl.Int32).clone()`.
- `cast()` is polars' equivalent of pandas' `astype()`.
- For a list of dtypes see the official [documentation](see https://pola-rs.github.io/polars/py-polars/html/reference/datatypes.html).



### Optimizing dtypes

Find the most efficient dtype for a series:

In [109]:
s.shrink_dtype()

a
i8
1
2
3


Also see [here](http://braaannigan.github.io/software/2022/10/31/polars-dtype-diet.html).

Shrink the memory allocation to the size of the actual data (in place).

In [110]:
s.shrink_to_fit() 

a
i64
1
2
3


## Ordering and sorting 


In [111]:
s.sort()

a
i64
1
2
3


In [112]:
s.reverse()

a
i64
3
2
1


In [113]:
s.rank()

a
f32
1.0
2.0
3.0


In [114]:
s.arg_sort() 

a
u32
0
1
2


`arg_sort()` returns the indices that would sort the series. Same as R's `order()`.


In [115]:
s.sort() == s[s.arg_sort()]

a
bool
true
true
true


`arg_sort()` can also be used to return the original series from the sorted one:

In [116]:
s == s[s[s.arg_sort()].arg_sort()]

a
bool
true
true
true


In [117]:
s.shuffle(seed=1)

a
i64
2
1
3


## Missing

Pandas users will be excited to know that polars has built in missing value support (!) for all dtypes.
This has been a long awaited feature in the Python data science ecosystem, with implications on performance and syntax.


In [118]:
m = pl.Series("a", [1, 2, None, np.nan])
m.is_null()

a
bool
false
false
true
false


In [119]:
m.is_nan()

a
bool
false
false
null
true


In [120]:
m1 = pl.Series("a", [1, None, 2, ]) # python native None
m2 = pl.Series("a", [1, np.nan, 2, ]) # numpy's nan
m3 = pl.Series("a", [1, float('nan'), 2, ]) # python's nan
m4 = pd.Series([1, None, 2 ])
m5 = pd.Series([1, np.nan, 2, ])
m6 = pd.Series([1, float('nan'), 2, ])

In [121]:
[m1.sum(), m2.sum(), m3.sum(), m4.sum(), m5.sum(), m6.sum()]

[3, nan, nan, 3.0, 3.0, 3.0]

Things to note:

- The use of `is_null()` instead of pandas `isna()`.
- Polars supports `np.nan` but that is a different dtype than `None` (which is a `Null` type). `None` is not considered 
- Aggregating pandas and polars series behave differently w.r.t. missing values:
  - Both will ignore `None`; which is unsafe.
  - Polars will not ignore `np.nan`; which is safe. Pandas is unsafe w.r.t. `np.nan`, and will ignore it. 


Filling missing values; `None` and `np.nan` are treated differently:

In [122]:
m1.fill_null(0)

a
i64
1
0
2


In [123]:
m1.interpolate()

a
i64
1
1
2


In [124]:
m2.fill_null(0)

a
f64
1.0
NaN
2.0


In [125]:
m2.fill_nan(0)

a
f64
1.0
0.0
2.0


In [126]:
m1.drop_nulls()

a
i64
1
2


In [127]:
m1.drop_nans()

a
i64
1
null
2


In [128]:
m2.drop_nulls()

a
f64
1.0
NaN
2.0


## Export


In [129]:
s.to_frame()

a
i64
1
2
3


In [130]:
s.to_list()

[1, 2, 3]

In [131]:
s.to_numpy()

array([1, 2, 3])

In [132]:
s.to_pandas()

0    1
1    2
2    3
Name: a, dtype: int64

In [133]:
s.to_arrow()

[
  1,
  2,
  3
]

## Date and Time

There are 4 datetime dtypes in polars:

1. Date: A date, without hours. Generated with `pl.Date()`.
2. Datetime: Date and hours. Generated with `pl.Datetime()`.
3. Duration: As the name suggests. Similar t o `timedelta` in pandas. Generated with `pl.Duration()`.
4. Time: Hour of day. Generated with `pl.Time()`.